In [108]:
import qiskit.quantum_info as qi
import constant
from qoop.core.random_circuit import generate_with_pool
from qoop.core.gradient import grad_loss
import splitter, utilities
%load_ext autoreload
%autoreload 2
num_qubits = 10
qc = generate_with_pool(num_qubits, 500)
qc = qc.assign_parameters([1] * qc.num_parameters)
# print(qc.draw())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [92]:
import qiskit, numpy as np
def split_control2(qc: qiskit.QuantumCircuit) -> list[qiskit.QuantumCircuit]: 
    qcs = []
    sub_qc = qiskit.QuantumCircuit(qc.num_qubits)
    counter = 0
    active_2qubit = 0 # 0 mean dactive, 1 mean active, 2 mean break instruction
    slots = np.zeros(qc.num_qubits)
    for _, qargs, _ in qc.data: 
        indices = utilities.get_qubit_indices(qargs)
        if len(indices) == 2:
            active_2qubit += 1
        
        slots = utilities.update_slot(slots, indices)
        if any(slot > 1 for slot in slots) or active_2qubit == 2:
            qcs.append(sub_qc)
            active_2qubit = 0
            sub_qc = qiskit.QuantumCircuit(qc.num_qubits)
            sub_qc.append(qc[counter][0], qc[counter][1])
            if len(indices) == 2:
                active_2qubit += 1
            slots = np.zeros(qc.num_qubits)
            slots = utilities.update_slot(slots, utilities.get_qubit_indices(qargs))
            qc.data = qc.data[counter + 1:]
            counter = 0
        else:
            sub_qc.append(qc[counter][0], qc[counter][1])
            counter += 1
            if counter >= len(qc.data):
                qcs.append(sub_qc)
                return qcs
    return qcs


In [111]:
qc = ansatz.Walltoall_zxz(4, 200)
qc = qc.assign_parameters([1] * qc.num_parameters)

In [112]:
qcs = split_control2(qc.copy())

In [113]:
qcs1 = splitter.split_control(qc.copy())

In [107]:
len(qcs1) == len(qcs)

True